# 01-Gaussian 解析与检查

使用 `AutoParser` 解析真实 Gaussian 日志测试文件，检查帧状态，并输出紧凑、可核对的摘要信息。

In [1]:
import os
from pathlib import Path

import pandas as pd

os.environ.setdefault("TQDM_DISABLE", "1")

from molop.io import AutoParser


def find_repo_root(start: Path | None = None) -> Path:
    current = (start or Path.cwd()).resolve()
    for parent in [current, *current.parents]:
        if (parent / "pyproject.toml").exists():
            return parent
    raise FileNotFoundError("Could not find repository root containing pyproject.toml")


repo_root = find_repo_root()
fixture_path = repo_root / "tests/test_files/g16log/2-TS1-Opt.log"
relative_fixture_path = fixture_path.relative_to(repo_root)

batch = AutoParser(str(fixture_path), n_jobs=1)
parsed_file = batch[0]
last_frame = parsed_file[-1]

print(f"Fixture: {relative_fixture_path.as_posix()}")
print(f"Number of files parsed: {len(batch)}")
print(f"Number of frames: {len(parsed_file)}")
print(f"Last frame is TS: {last_frame.is_TS}")
print(f"Last frame is normal: {last_frame.is_normal}")
print(f"Last frame has error: {last_frame.is_error}")

energy_quantity = getattr(getattr(last_frame, "energies", None), "total_energy", None)
energy_hartree = None
if energy_quantity is not None:
    try:
        energy_hartree = round(float(str(energy_quantity).split()[0]), 6)
    except (TypeError, ValueError, IndexError):
        try:
            energy_hartree = round(float(energy_quantity), 6)
        except (TypeError, ValueError):
            energy_hartree = None

summary_df = batch.to_summary_df()
print(f"summary_df.shape: {summary_df.shape}")

snippet_df = pd.DataFrame(
    [
        {
            "fixture": relative_fixture_path.as_posix(),
            "num_frames": len(parsed_file),
            "is_ts": bool(last_frame.is_TS),
            "is_normal": bool(last_frame.is_normal),
            "is_error": bool(last_frame.is_error),
            "total_energy_hartree": energy_hartree,
        }
    ]
)
print("summary_snippet:")
print(snippet_df.head(1).to_string(index=False))


Fixture: tests/test_files/g16log/2-TS1-Opt.log
Number of files parsed: 1
Number of frames: 106
Last frame is TS: False
Last frame is normal: True
Last frame has error: False


summary_df.shape: (1, 21)
summary_snippet:
                              fixture  num_frames  is_ts  is_normal  is_error  total_energy_hartree
tests/test_files/g16log/2-TS1-Opt.log         106  False       True     False          -3667.861749
